In [4]:
import dataclasses
import logging
import math
import os
import io
import sys
import time
import json
import random
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, DebertaV2ForSequenceClassification
import torch
from tqdm import tqdm

In [6]:
reward_name = "/home/tiger/.cache/huggingface/hub/models--OpenAssistant--reward-model-deberta-v3-large-v2/snapshots/c355404efa9ad2ad069f3a197cae0523c14244fc"
model: DebertaV2ForSequenceClassification = AutoModelForSequenceClassification.from_pretrained(reward_name)
print(model)
print(type(model))

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [ ]:
model.forward()

In [2]:
with open("/opt/tiger/MoDS/quality-evaluation/input.json", "r") as f:
    data = f.read()
    data = json.loads(data)
print(len(data))

52002


In [3]:
data = json.dumps(data[:1000])
with open("/opt/tiger/MoDS/quality-evaluation/data.json", "w") as f:
    f.write(data)

In [2]:
from vllm import LLM

In [6]:
model = LLM(model="gpt2", tensor_parallel_size=6, dtype="float16")
print(model)

/home/tiger/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-26 17:30:16,005	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


In [1]:
from datasets import load_dataset

dataset = load_dataset("TIGER-Lab/MathInstruct", cache_dir="/mnt/bn/data-tns-live-llm/leon/datasets/")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
import polars as pl
df = pl.read_parquet("/opt/tiger/MoDS/code.parquet")

In [5]:
print(df.head())

shape: (5, 3)
┌──────────────────────────────────┬─────────────────────────────┬─────────────────────────────────┐
│ instruction                      ┆ input                       ┆ output                          │
│ ---                              ┆ ---                         ┆ ---                             │
│ str                              ┆ str                         ┆ str                             │
╞══════════════════════════════════╪═════════════════════════════╪═════════════════════════════════╡
│ Create a function that takes a…  ┆                             ┆ def f(x):                       │
│                                  ┆                             ┆     """                         │
│                                  ┆                             ┆     Takes a …                   │
│ Generate a unique 8 character …  ┆                             ┆ import string                   │
│                                  ┆                             ┆ import ran

In [6]:
data = pl.read_json("/opt/tiger/MoDS/MathInstruct.json")
print(data.head())

shape: (5, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ source                         ┆ instruction                    ┆ output                         │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ str                            ┆ str                            │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ data/CoT/aqua_rat.json         ┆ The distance between two       ┆ Let's think about the          │
│                                ┆ stars…                         ┆ multi-ch…                      │
│ data/CoT/aqua_rat.json         ┆ How many ways can the letters  ┆ Let's solve the multi-choice   │
│                                ┆ …                              ┆ q…                             │
│ data/PoT/aqua_rat_filtered.jso ┆ A team of six entered for a    ┆ answers =

In [8]:
import torch
t1 = torch.load("/opt/tiger/Cherry_LLM/code_data_pre.pt")
print(type(t1))

<class 'list'>


In [9]:
print(t1[0])

{'ppl': [tensor(18.6185), 0, 0], 'sent_emb': [tensor([[-0.3425,  0.0562,  0.7246,  ..., -0.0523, -0.9365, -1.1855]],
       dtype=torch.float16), 0, 0]}


In [1]:
from datasets import load_dataset
dataset = load_dataset("parquet", data_files = "/mnt/bn/data-tns-live-llm/leon/datasets/code.parquet", split = "train")

In [4]:
print(dataset[0])

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.', 'input': '', 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}


In [6]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [7]:
def format_function(input):
    prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
    sources = prompt_input.format_map(input) if input.get("input", "") != "" else prompt_no_input.format_map(input)
    return sources

In [8]:
print(format_function(dataset[0]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.

### Response:


In [5]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/pre-exp-code/")
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)


In [12]:
import torch
data = torch.load("/mnt/bn/data-tns-live-llm/leon/datasets/code_data_cherry.pt")
ifd = [d["ppl"][2]/d["ppl"][1] for d in data]
idxs = sorted(range(len(ifd)), key=lambda i: ifd[i], reverse=True)
print(idxs)

[765, 681, 913, 129, 740, 723, 178, 64, 44, 176, 631, 774, 15, 38, 763, 668, 688, 847, 911, 239, 515, 333, 999, 147, 363, 956, 150, 501, 100, 871, 732, 619, 143, 507, 460, 582, 939, 449, 743, 691, 895, 482, 404, 706, 648, 179, 383, 949, 433, 898, 751, 824, 104, 189, 845, 802, 767, 496, 59, 352, 458, 931, 237, 987, 108, 141, 393, 271, 961, 394, 354, 645, 320, 519, 476, 781, 474, 524, 297, 980, 111, 726, 362, 154, 534, 166, 928, 874, 251, 351, 943, 970, 463, 359, 79, 759, 210, 235, 993, 710, 95, 741, 160, 803, 177, 967, 646, 35, 162, 203, 68, 385, 266, 731, 39, 784, 622, 806, 537, 677, 219, 54, 771, 872, 445, 83, 279, 854, 437, 312, 133, 841, 637, 690, 309, 825, 371, 343, 248, 300, 918, 296, 479, 483, 174, 84, 698, 530, 960, 563, 531, 594, 332, 651, 849, 535, 811, 926, 148, 114, 185, 369, 708, 71, 888, 838, 687, 953, 983, 288, 764, 823, 842, 41, 258, 709, 738, 262, 873, 817, 165, 466, 117, 770, 403, 254, 459, 650, 103, 951, 470, 402, 589, 954, 810, 627, 684, 497, 278, 163, 391, 596, 378,

In [15]:
import json
with open("/mnt/bn/data-tns-live-llm/leon/datasets/code-quality-evaluation.json", "r") as f:
    d1 = json.loads(f.read())
scores = [d["reward_score"] for d in d1[:1000]]
sidxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
print(sidxs)

[413, 966, 127, 357, 836, 200, 210, 947, 72, 377, 790, 473, 130, 674, 622, 972, 653, 594, 266, 416, 219, 655, 639, 654, 93, 757, 970, 404, 656, 358, 288, 106, 209, 151, 815, 49, 348, 513, 984, 732, 948, 96, 269, 501, 279, 778, 102, 994, 579, 478, 569, 608, 826, 687, 104, 811, 50, 399, 591, 927, 807, 254, 487, 647, 382, 388, 83, 201, 481, 849, 694, 548, 842, 679, 101, 759, 276, 318, 531, 466, 894, 573, 722, 690, 954, 511, 902, 781, 211, 431, 296, 35, 89, 981, 983, 658, 19, 714, 820, 597, 710, 439, 55, 999, 527, 890, 664, 542, 546, 483, 297, 395, 944, 528, 384, 22, 635, 910, 265, 270, 207, 977, 249, 470, 33, 274, 312, 574, 817, 832, 16, 969, 975, 758, 870, 66, 955, 568, 138, 614, 262, 878, 489, 88, 328, 562, 630, 610, 345, 823, 621, 551, 491, 691, 967, 461, 323, 187, 797, 800, 899, 595, 686, 672, 107, 818, 673, 289, 427, 194, 719, 728, 779, 843, 803, 612, 976, 845, 44, 746, 257, 13, 667, 361, 903, 195, 452, 925, 834, 907, 480, 602, 449, 286, 225, 99, 352, 373, 3, 880, 841, 152, 223, 240,

In [16]:
print(scores[413])

8.421875
